<a href="https://colab.research.google.com/github/nhwagle/code-match/blob/main/Objective_1_Equation_To_Code_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file contains code we utilized with a simple contrastive loss function to attempt to predict when code and equations went together. We find that our models immediately shortcut to the random chance solution, and do not further work to align text embeddings between our latex model and the TransCoder model.

### Imports

Installing codegen. Their install_env.sh won't work for colab so modifying such that all conda imports are done with pip

In [ ]:
%%capture
%pip install transformers pylatexenc

In [ ]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from transformers import DistilBertTokenizer, DistilBertModel
from pylatexenc.latex2text import LatexNodes2Text

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive/RSS/Data

algorithms.csv	code.csv	  code-small.csv  func-rep-small.csv
code-all.csv	code-func-rep.pt  equations.csv   func-rep-small.pt


In [ ]:
eqns = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/equations.csv")
algs = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/algorithms.csv")
code = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/code.csv")

Comparing methods for reading in latex

In [ ]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertModel.from_pretrained("distilbert-base-cased")


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
eqn_index = random.randint(0,10000)
#bellman:
# eqn_index = 3210


print(f"Example of equation index number {eqn_index}")

latex = eqns.iloc[eqn_index].eqn
print("")
print("======")
print("Multistep")
print("======")
print("Raw latex below:")
print("")
print(latex)
print(len(latex))
print("Converted Unicode below:")
uni_rep = LatexNodes2Text().latex_to_text(latex)
print(uni_rep)
print("Tokenization below:")
print("")
print(tokenizer(uni_rep)['input_ids'])
print("")
print("Inverted Tokenization")
print("")
print(tokenizer.decode(tokenizer(uni_rep)['input_ids']))

print("======")
print("Single step")
print("======")
print("Inverted Tokenization")
print("")
print(tokenizer.decode(tokenizer(latex)['input_ids']))

Example of equation index number 1045

Multistep
Raw latex below:

 \label{def-twc}
tw\left(x,s\right) = \frac{\Sigma_{p \in R\left(x\right)} \mid  \{c \mid s \models c   
    \wedge positiveImpact\left (p, c \right) 
    \wedge p \in s \wedge active\left(x,p \right)\}\mid 
%\quad \textnormal{and} \quad
%twc^-\left(x,s\right) = \Sigma_{p \in R\left(x\right)} \mid\{c \mid neg\left(x,p,c,s\right)\}\mid \quad \quad
}{
\Sigma_{p \in R\left(x\right)} \mid \{c \mid \left(s \not\models c \vee \neg positiveImpact\left (p, c \right)\right) \wedge addBy\left (c,p \right) \wedge p \in s \wedge active\left(x,p \right) \}\mid \textnormal{+} 1
}

573
Converted Unicode below:
 
tw(x,s) = Σ_p ∈R(x) |{c |s c   
    ∧positiveImpact(p, c ) 
    ∧p ∈s ∧active(x,p )}|

/
Σ_p ∈R(x) |{c |(s c ∨positiveImpact(p, c )) ∧addBy(c,p ) ∧p ∈s ∧active(x,p ) }|+ 1


Tokenization below:

[101, 189, 2246, 113, 193, 117, 188, 114, 134, 408, 168, 185, 850, 2069, 113, 193, 114, 197, 196, 172, 197, 188, 172, 856, 5674, 2247

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
import gensim.downloader
print(latex)
word_tokenize(latex)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
\label{NstepOptimBellman}
 Q^*(s_0, a_0) = \E_{\Traj_{:N} \sim \pi^* \mid s_0, a_0} \left[ \sum_{t=0}^{N-1} \gamma^{t}r_t + \gamma^N \E_{s_N} \max_{a^*_N} Q^*(s_N, a^*_N) \right]   



['\\label',
 '{',
 'NstepOptimBellman',
 '}',
 'Q^*',
 '(',
 's_0',
 ',',
 'a_0',
 ')',
 '=',
 '\\E_',
 '{',
 '\\Traj_',
 '{',
 ':',
 'N',
 '}',
 '\\sim',
 '\\pi^*',
 '\\mid',
 's_0',
 ',',
 'a_0',
 '}',
 '\\left',
 '[',
 '\\sum_',
 '{',
 't=0',
 '}',
 '^',
 '{',
 'N-1',
 '}',
 '\\gamma^',
 '{',
 't',
 '}',
 'r_t',
 '+',
 '\\gamma^N',
 '\\E_',
 '{',
 's_N',
 '}',
 '\\max_',
 '{',
 'a^*_N',
 '}',
 'Q^*',
 '(',
 's_N',
 ',',
 'a^*_N',
 ')',
 '\\right',
 ']']

In [ ]:
%matplotlib inline

## Standard Library
import random
from string import punctuation
from collections import Counter

## External Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
import gensim.downloader

# PyTorch Modules: see http://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
print("GPU Available:", torch.cuda.is_available())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
GPU Available: True


Appears to be how to work with tokenized latex...

In [ ]:
model(**tokenizer(latex,return_tensors='pt',truncation=True))[0].shape

torch.Size([1, 125, 768])

In [ ]:
test = tokenizer(latex,return_tensors='pt',truncation=True)
test

{'input_ids': tensor([[  101,   165,  3107,   196,   151, 13894,  1643,  2346,  6451,  4060,
          2064,  3991,  1399,   198,   154,   167,   115,   113,   188,   168,
           121,   117,   170,   168,   121,   114,   134,   165,   142,   168,
           196,   165,   157, 18663,   168,   196,   131,   151,   198,   165,
         27466,  1306,   165,   185,  1182,   167,   115,   165,  2286,   188,
           168,   121,   117,   170,   168,   121,   198,   165,  1286,   164,
           165,  7584,   168,   196,   189,   134,   121,   198,   167,   196,
           151,   118,   122,   198,   165, 21400,   167,   196,   189,   198,
           187,   168,   189,   116,   165, 21400,   167,   151,   165,   142,
           168,   196,   188,   168,   151,   198,   165, 12477,  1775,   168,
           196,   170,   167,   115,   168,   151,   198,   154,   167,   115,
           113,   188,   168,   151,   117,   170,   167,   115,   168,   151,
           114,   165,  1268,   166,  

In [ ]:
class ContrastiveModel(nn.Module):
  def __init__(self, embedding_model):
    super(ContrastiveModel,self).__init__()

    self.latex_encoder = embedding_model

    self.latex_linear1 = nn.Linear(768,1024)
    self.linear_comb1 = nn.Linear(1024,1)
    # self.code_linear1 = nn.Linear(1024,900)
    # self.code_linear2 = nn.Linear(900,768)

    self.relu = nn.ReLU()

    # self.linear_comb1 = nn.Linear(768,256)
    # self.linear_comb2 = nn.Linear(256,64)
    # self.linear_comb3 = nn.Linear(64,1)
    

  def forward(self, embedded_functions, in_ids,atten_mask):
      # out1 = self.code_linear1(embedded_functions)
      # out2 = self.code_linear2(self.relu(out1))
      # embedded_funcs = self.relu(out2)

      embedded_latex = self.relu(self.latex_encoder(input_ids=in_ids,attention_mask=atten_mask)[0].mean(dim=1))
      upscaled_latex = self.latex_linear1(embedded_latex)

      combined_output = self.relu(embedded_functions*upscaled_latex)

      return self.linear_comb1(combined_output)
      # comb_out1 = self.relu(self.linear_comb1(combined_output))
      # comb_out2 = self.relu(self.linear_comb2(comb_out1))
      # comb_out3 = self.linear_comb3(comb_out2)
      
      # return comb_out3#.max(-2)

In [ ]:
!ls gdrive/MyDrive/RSS/Data

algorithms.csv	code.csv	  code-small.csv  func-rep-small.csv
code-all.csv	code-func-rep.pt  equations.csv   func-rep-small.pt


In [ ]:
func_rep_df = pd.read_pickle('gdrive/MyDrive/RSS/Data/code-func-rep.pt')

In [ ]:
algs.id.unique()

,Unnamed: 0,id,alg
0,0,2201.02779,[t!]\n\n \SetKwFunction{FMain}{$DGLSegment$}\n...
1,1,2201.03211,[ht]\n\begin{flushleft}\n\caption{Proposed NMC...
2,2,2201.03211,[tb]\n\begin{flushleft}\n\caption{Proposed NMC...
3,3,2201.03156,[H]\n\t\begin{algorithmic}[1]\n\t\t\caption{Au...
4,4,2201.03156,[H]\n\t\begin{algorithmic}[1]\n\t\t\caption{Tr...
...,...,...,...
6785,1334,1806.03085,[H]\n\caption{One iteration of the Stein varia...
6786,1335,1709.04347,\n%\t\caption{\\Stacked Zoom Out-and-In Net wi...
6787,1336,1709.04347,\n%\t\caption{Zoom Out-and-In Network with Map...
6788,1337,1711.01244,"[H] \n\t\caption{MLAP algorithm, meta-training..."


In [ ]:
len(algs.id.unique())

3254

In [ ]:
len(eqns.id.unique())

7472

In [ ]:
func_rep_df = func_rep_df[func_rep_df.functions.apply(len) != 0]
func_rep_df

,id,functions,representations
0,1811.02182,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
1,1811.02182,"[def check_config_used ( config , target_sourc...","[[[tensor(0.6108), tensor(-0.2560), tensor(0.1..."
2,1811.02182,"[def __init__ ( self , batch_size , paired = F...","[[[tensor(0.8430), tensor(0.1034), tensor(0.30..."
3,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
4,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
...,...,...,...
29193,2112.01073,"[def my_lcs ( string , sub ) : NEW_LINE INDENT...","[[[tensor(0.6292), tensor(0.0059), tensor(0.09..."
29194,2112.01073,"[def __init__ ( self , n = 4 ) : NEW_LINE INDE...","[[[tensor(0.4932), tensor(-0.1197), tensor(-0...."
29195,2112.01073,"[def precook ( s , n = 4 , out = False ) : NEW...","[[[tensor(0.7558), tensor(-0.0280), tensor(0.0..."
29197,2112.01073,[def __init__ ( self ) : NEW_LINE INDENT self ...,"[[[tensor(0.7251), tensor(-0.0360), tensor(0.0..."


In [ ]:
eqns['id'] = eqns.id.astype(str)
func_rep_df['id'] = func_rep_df['id'].astype(str)
collected_func_reps = func_rep_df.groupby(by='id').sum()

In [ ]:
func_rep_df

,id,functions,representations
0,1811.02182,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
1,1811.02182,"[def check_config_used ( config , target_sourc...","[[[tensor(0.6108), tensor(-0.2560), tensor(0.1..."
2,1811.02182,"[def __init__ ( self , batch_size , paired = F...","[[[tensor(0.8430), tensor(0.1034), tensor(0.30..."
3,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
4,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
...,...,...,...
29193,2112.01073,"[def my_lcs ( string , sub ) : NEW_LINE INDENT...","[[[tensor(0.6292), tensor(0.0059), tensor(0.09..."
29194,2112.01073,"[def __init__ ( self , n = 4 ) : NEW_LINE INDE...","[[[tensor(0.4932), tensor(-0.1197), tensor(-0...."
29195,2112.01073,"[def precook ( s , n = 4 , out = False ) : NEW...","[[[tensor(0.7558), tensor(-0.0280), tensor(0.0..."
29197,2112.01073,[def __init__ ( self ) : NEW_LINE INDENT self ...,"[[[tensor(0.7251), tensor(-0.0360), tensor(0.0..."


In [ ]:
collected_func_reps

,functions,representations
id,,
1010.1763,"[def robust_nmf ( data , rank , beta , init , ...","[[[tensor(0.8621), tensor(-0.1790), tensor(0.4..."
1110.2997,[def loglikelihood ( theta ) : NEW_LINE INDENT...,"[[[tensor(0.8383), tensor(0.0016), tensor(0.13..."
1212.6094,[def main ( ) : NEW_LINE INDENT cleaned_model_...,"[[[tensor(0.5931), tensor(-0.0021), tensor(0.3..."
1604.02071,"[def __init__ ( self , products_set_path , rat...","[[[tensor(0.3454), tensor(-0.0555), tensor(-0...."
1711.05114,[def exe_time ( func ) : NEW_LINE INDENT def n...,"[[[tensor(0.6678), tensor(-0.0102), tensor(0.2..."
...,...,...
2112.01529,[def readme ( ) : NEW_LINE INDENT with open ( ...,"[[[tensor(0.7760), tensor(-0.0030), tensor(0.1..."
2112.0153,[def main ( opt ) : NEW_LINE INDENT uvs = [ jo...,"[[[tensor(0.5144), tensor(-0.2040), tensor(0.2..."
711.0962,"[def __init__ ( self , name = None , opts = No...","[[[tensor(0.6273), tensor(-0.0820), tensor(0.2..."


In [ ]:
23814/777

30.64864864864865

In [ ]:
np.mean(collected_func_reps.functions.apply(len))

242.06177606177607

In [ ]:
demofuncs = collected_func_reps[collected_func_reps.index=='1811.02182']
demofuncs.functions

id
1811.02182    [def main ( config ) : NEW_LINE INDENT from da...
Name: functions, dtype: object

In [ ]:
demofuncs.functions[0]

["def main ( config ) : NEW_LINE INDENT from data_loader import DataLoader NEW_LINE if ( config . trainer == ' minimize _ DCE ' ) : NEW_LINE INDENT from trainer_DCE import Trainer NEW_LINE paired = True NEW_LINE DEDENT elif ( config . trainer == ' acoustic _ supervision ' ) : NEW_LINE INDENT from trainer_acoustic import Trainer NEW_LINE paired = False NEW_LINE DEDENT elif ( config . trainer == ' AAS ' ) : NEW_LINE INDENT from trainer_AAS import Trainer NEW_LINE paired = False NEW_LINE DEDENT elif ( config . trainer == ' FSEGAN ' ) : NEW_LINE INDENT from trainer_FSEGAN import Trainer NEW_LINE paired = True NEW_LINE DEDENT if config . gpu >= 0 : NEW_LINE INDENT torch . cuda . manual_seed ( config . random_seed ) NEW_LINE torch . cuda . set_device ( config . gpu ) NEW_LINE DEDENT if ( config . DB_name == ' librispeech ' ) : NEW_LINE INDENT if ( paired ) : NEW_LINE INDENT config . tr_ny_manifest = ' data / libri _ tr _ ny _ paired . csv ' NEW_LINE config . trsub_manifest = ' data / libri _

In [ ]:
full_df = eqns.merge(collected_func_reps , on='id')

In [ ]:
full_df

,Unnamed: 0,id,eqn,functions,representations
0,17744,1811.02182,\n\begin{split}\n \min_{G}\max_{D} V_{cGA...,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
1,17745,1811.02182,\n\begin{split}\n \min_{G}\max_{D}V_{upcG...,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
2,17746,1811.02182,"\n L_{CTC}(E) = -\E_{(\textbf{m}, \textb...",[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
3,17747,1811.02182,\n\t\tp(\textbf{t}|E(\textbf{m})) = \sum_{\te...,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
4,17748,1811.02182,\n\begin{split}\n \min_{E}\max_{D}V_{upcB...,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
...,...,...,...,...,...
5420,25341,2112.01073,"\n\small\nP(w_t|w_{\textless t}, V, S;\theta) ...","[def extract_frames ( video , dst ) : NEW_LINE...","[[[tensor(0.7743), tensor(-0.1384), tensor(0.0..."
5421,25342,2112.01073,\n\small\n\setlength{\abovedisplayskip}{5pt}\n...,"[def extract_frames ( video , dst ) : NEW_LINE...","[[[tensor(0.7743), tensor(-0.1384), tensor(0.0..."
5422,25343,2112.01073,\n\small\n\setlength{\abovedisplayskip}{5pt}\n...,"[def extract_frames ( video , dst ) : NEW_LINE...","[[[tensor(0.7743), tensor(-0.1384), tensor(0.0..."
5423,25344,2112.01073,\n\small\n\setlength{\abovedisplayskip}{3pt}\n...,"[def extract_frames ( video , dst ) : NEW_LINE...","[[[tensor(0.7743), tensor(-0.1384), tensor(0.0..."


In [ ]:
for row in full_df.iterrows():
  print(tokenizer(row[1].eqn,return_tensors='pt',truncation=True))
  break

{'input_ids': tensor([[  101,   165,  3295,   196,  3325,   198,   165, 11241,   168,   196,
           144,   198,   165, 12477,  1775,   168,   196,   141,   198,   159,
           168,   196,   172, 10583,  2249,   198,   113,   144,   117,   141,
           114,   134,   165,   142,   168,   196,   113,   165,  3087,  1830,
          2087,   196,   193,   198,   168,   196,   188,   198,   117,   165,
          3087,  1830,  2087,   196,   193,   198,   168,   196,   189,   198,
           114,   165, 27466,  1306,   185,   113,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   196,   188,   198,   117,   165,  3087,
          1830,  2087,   196,   193,   198,   168,   196,   189,   198,   114,
           198,   164,   165,  9366,   141,   113,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   188,   117,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   196,   189,   198,   114,   166,   165,
           165,   116,   165,   142,  

In [ ]:
class ContrastiveDataset(Dataset):

    def __init__(self, combined_df):
        """Simple init function"""
        self.df = combined_df
        tok_eqns = []
        for row in self.df.iterrows():
          tok_eqns.append(tokenizer(row[1].eqn,return_tensors='pt',truncation=True))
        self.tok_eqns = tok_eqns


    def __len__(self):
        """Get length of dataset, doubled from our data as we will do 50/50 positive and negative examples"""
        return len(self.df) * 2


    def __getitem__(self, idx):
        "Contrastive learning and makes half negative samples half positive"

        if idx < len(self.df):
            label = 1
            entry = self.df.iloc[idx]
            eqn_paper_id = entry.id
            code_id = eqn_paper_id
            code_rep = entry.representations
            code_funcs = entry.functions
            eqn = entry.eqn
            tok_eqn = self.tok_eqns[idx]

        else:
            eqn_idx = idx - len(self.df)
            tok_eqn = self.tok_eqns[eqn_idx]
            label = 0

            curr_code_idx = random.randint(0,len(self.df)-1)
            curr_paper = self.df.iloc[curr_code_idx].id 
            while (curr_paper == self.df.iloc[eqn_idx].id):
                curr_code_idx = random.randint(0,len(self.df)-1)
                curr_paper = self.df.iloc[curr_code_idx].id
            
            code_id = curr_paper
            eqn_entry = self.df.iloc[eqn_idx]
            code_entry = self.df.iloc[curr_code_idx]

            eqn_paper_id = eqn_entry.id 
            code_rep = code_entry.representations 
            code_funcs = code_entry.functions 
            eqn = eqn_entry.eqn

        return code_rep,tok_eqn, label, eqn_paper_id,code_id,eqn, code_funcs


In [ ]:
train_dataset = ContrastiveDataset(full_df)

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)

In [ ]:
model = DistilBertModel.from_pretrained("distilbert-base-cased")

combined_model = ContrastiveModel(model)
combined_model.train()
combined_model.cuda()

optimizer = optim.Adam(combined_model.parameters(), lr=.01)
criterion = nn.BCEWithLogitsLoss()
epochs = 5



Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!nvidia-smi

Tue Apr 26 15:41:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    42W / 250W |   4649MiB / 16280MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
epochs = 20

In [ ]:
for epoch in range(epochs):
    correct = 0
    total = 0
    loss_avg = 0
    for index,input in enumerate(train_loader):

      if index % 100 == 0:
        print(loss_avg/100)
        loss_avg = 0
        print(f"On batch {index}")
        print(correct/(total+1))

      code_rep, tok_eqn, label, eqn_paper_id,code_id,eqn, code_funcs = input
      orig_label = label

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      curr_rep = torch.cat(code_rep).cuda()
      input_ids,a_mask = tok_eqn['input_ids'].flatten(1).cuda(),tok_eqn['attention_mask'].flatten(1).cuda()
      label = torch.tensor(label,dtype=float,device='cuda')

      logits = combined_model(curr_rep,input_ids,a_mask)
      curr_logit = torch.mean(logits)

      loss = criterion(curr_logit.reshape((1)),label)
      loss.backward()
      optimizer.step()

      loss_avg += loss.item()
      pred = torch.sigmoid(curr_logit).item()
      if round(pred) == orig_label:
          correct += 1
      total += 1
      print(pred)
    print("----")

    print(f"Epoch {epoch} correct {correct}, total {total}, {loss_avg/50}")
    print("----")
        # outputs = net(inputs)
        # loss = criterion(outputs, labels)
        # loss.backward()
        # optimizer.step()

        # # print statistics
        # running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print(f'[{epoch + 1}, {i + 

0.0
On batch 0
0.0
0.4907546043395996
0.4938477873802185
0.4946592152118683
0.49264445900917053


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.49273890256881714
0.48203763365745544
0.48144516348838806
0.5051873922348022
0.49197623133659363
0.49131083488464355
0.5029921531677246
0.4919959604740143
0.4892311096191406
0.49202948808670044
0.4918978810310364
0.4891364574432373
0.4892519414424896
0.48951083421707153
0.47943076491355896
0.49294668436050415
0.4919142723083496
0.5051082968711853
0.4925523102283478
0.492265522480011
0.4949178397655487
0.4837900698184967
0.48476576805114746
0.49491164088249207
0.5077707171440125
0.5085858106613159
0.4972054660320282
0.5103797912597656
0.49911677837371826
0.5119848847389221
0.501926064491272
0.5134442448616028
0.5006372928619385
0.502032995223999
0.5003935098648071
0.5007104873657227
0.5029435753822327
0.5147665739059448
0.5011324882507324
0.5019500851631165
0.5167108178138733
0.5175158977508545
0.5178390145301819
0.5035931468009949
0.5062186121940613
0.5051178932189941
0.5041821599006653
0.5045703053474426
0.5052367448806763
0.5061519145965576
0.5092331171035767
0.5233333706855774
0.5

NameError: ignored

In [ ]:
print(tok_eqn)

{'input_ids': tensor([[[  101,   182,   168,   189,   134,   176,   168,   189,   117,   165,
            186, 13284,  1181,   159,   168,   189,   134,   165,   175, 19366,
            196,   122,   198,   196,   189,   198,   165, 12523,  9019,   196,
           4267,  8517,   198,   165,  1286,   113,   165,  7584,   168,   196,
            178,   134,   122,   198,   167,   189,   176,   168,   178,   176,
            168,   178,   167,   196,   165,  1499,   198,   165,  1268,   114,
            119,   102]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]])}


In [ ]:
combined_model.latex_encoder(**tok_eqn)

ValueError: ignored

In [ ]:
demo = torch.randn((2,4,1024))